In [5]:

from citations.CitationClassifier import MultiCiteClassifier
from citations.Agglomerator import RankedClassificationCounts
from citations.FoundationModel import FoundationModel
from documents.Corpus import Corpus
import json, pickle
import warnings, logging
from datetime import datetime
import argparse
from config import FOUNDATION_MODELS_PATH, MARKDOWN_FILES_PATH, CITATION_MODEL_PATH, OPEN_ACCESS_PAPER_XREF
from utils.functional import extract_paper_metadata
import pandas as pd

In [3]:
classifier = MultiCiteClassifier(CITATION_MODEL_PATH)

corpus = Corpus(MARKDOWN_FILES_PATH, 
                    extensions = ['mmd'], 
                    paper_years=extract_paper_metadata(OPEN_ACCESS_PAPER_XREF),
                    foundation_model_limit=3
                    )

models = FoundationModel.modelsFromJSON(FOUNDATION_MODELS_PATH)
    
corpus.findAllPaperRefsAllTitles(models = models,
                                    classifier = classifier)

100%|██████████| 3/3 [00:00<00:00,  4.10it/s]


In [6]:
textualRefs = corpus.getAllTextualReferences(as_dict = True)
df = pd.DataFrame.from_dict(textualRefs)

In [44]:
df['classification_ranking'] = df.groupby(['modelKey', 'paperId'])['classification_order'].rank(method='min')
as_json = (df[df['classification_ranking'] == 1]
        .groupby(['modelKey', 'paperYear', 'classification'])['paperId']
        .nunique()
        .reset_index()
        .rename(columns={'paperId':'count'})
        .pivot(index=['modelKey', 'paperYear'], columns='classification', values='count')
        .fillna(0)
).to_json(orient = 'index')
print(as_json)

{"('1157_igpt-xl', 2023.0)":{"background":4,"uses":1},"('933_image-to-image_cgan', 2023.0)":{"background":1,"uses":1}}


In [4]:
from os import listdir
markdown_path = '/Users/Alex/Desktop/paper_analysis_toolkit/data/Markdown'
markdown_files = list(filter(lambda s: '.mmd' in s, listdir(markdown_path)))

with open('../data/open_access_paper_data.jsonl', 'r') as f:
    content = f.read()
    missing_files = [file for file in markdown_files if (file.split('.')[0]) not in content]

print(missing_files)


['447bbdbeb5dfa9252b51a833eafe5e8f4d3b632e.mmd', '2e92b3699668f920a8d692535622ebeaa53315e2.mmd', '5e748b212147681fd366eee9c9da13b956f5f577.mmd', '1673b72a4ebb56ae3775131937c40afae3853e9b.mmd', '4a806670b0aeb2b55c1efce5ae294e34ac9c676b.mmd', '46826780f57c8f3d5f299ed2a6e11a91fca746ee.mmd', 'bb8f7fbec020675d269ccfa0e6e603f02b664c0d.mmd', 'd0edff5402edeab721b7681643e1ff7c2354de4a.mmd', '0042c4f35d4a3ea947de8374e298099d476dad7f.mmd', 'e58b0ee9a1fdb15a72ee721053df3569127cde42.mmd', '7693ade76f4aeb9d04d816948fe2e79cd18b4ab2.mmd', '1b31882e60aaae3ac696e4f24f5cd93275c591f7.mmd', '7679dc8534cb1dd65c63c50b38f56386228d32d1.mmd', 'e476b207fffb2b15f0adc39f8f8c1c8643a63311.mmd', '74f614d1130f0bc50db3a54abe0d85eefaad894b.mmd', '0cb697d007997664264857311c5615e9b383b739.mmd', '87e26319485793e9cef3c5461b914b7f646a6162.mmd', '20cb4e0bd8871d33d82fc72ea82a0aa1dd922810.mmd', '605653eec16a75a0bd871da7d4f7eeed0620bac6.mmd', '7ee9301f239556896403e00404db3cfef3b60903.mmd', '7c4f6fd4c7eadcc7189a6797db215895340f93

In [5]:
len(missing_files)

920

In [ ]:

        classification_counts = (df[df['classification_ranking'] == 1]
                                                .groupby([group_key, classification_col])[additional_key]
                                                .nunique()
                                                .reset_index()
                                                .rename(columns={additional_key:'count'})
                                                .pivot(index=group_key, columns=classification_col, values='count')
                                                .fillna(0)
                                                .rename_axis(None, axis = 1)
                                                .reset_index()
                                                .set_index(group_key)
                                            )
        return classification_counts